# Задачи к Лекции 2

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv("/content/drive/MyDrive/mlbootcamp5_train.csv",
                 sep=";",
                 index_col="id")
df.head()

Mounted at /content/drive


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


## Задачи

**1. Построить наивный байесовский классификатор для количественных полей age, height, weight, ap_hi, ap_lo. Исправить данные, если это необходимо. Привести матрицу неточностей и сравнить со значением полученным в ходе лекции. Попытаться объяснить разницу.**

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix

# Предобработка данных и очистка
df['age'] = df['age'] / 365  # переводим возраст в годы
df = df[(df['ap_hi'] > 0) & (df['ap_lo'] > 0) & (df['ap_hi'] >= df['ap_lo'])]  # убираем очевидные ошибки

X = df[['age', 'height', 'weight', 'ap_hi', 'ap_lo']]
y = df['cardio']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

# Матрица неточностей
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

[[9301 1186]
 [6163 3974]]


**Комментарии:** Ваши комментарии здесь.

**2. Написать свой наивный байесовский классификатор для категориальных полей cholesterol, gluc. Привести матрицу неточностей и сравнить со значениями из задачи 1 (нельзя использовать готовое решение из sklearn) (не обязательно)**

In [4]:
def naive_bayes_categorical(X, y):
    unique_classes = np.unique(y)
    predictors = X.columns
    prob_data = {label:{} for label in unique_classes}

    for label in unique_classes:
        subset = X[y == label]
        total = len(subset)
        for predictor in predictors:
            prob_data[label][predictor] = {}
            counts = subset[predictor].value_counts().to_dict()
            for key, count in counts.items():
                prob_data[label][predictor][key] = count / total

    def predict(x):
        probs = {}
        for label in unique_classes:
            p = 1
            for predictor in predictors:
                p *= prob_data[label][predictor].get(x[predictor], 0)
            probs[label] = p
        return max(probs, key=probs.get)

    return predict

X_cat = df[['cholesterol', 'gluc']]
y = df['cardio']
pred_func = naive_bayes_categorical(X_cat, y)
X_train, X_test, y_train, y_test = train_test_split(X_cat, y, test_size=0.3, random_state=17)
predictions = X_test.apply(pred_func, axis=1)
conf_mat = confusion_matrix(y_test, predictions)
print(conf_mat)


[[8262 2225]
 [6231 3906]]


**Комментарии:** Ваши комментарии здесь.

**3. Построить наивный байесовский классификатор для бинарных полей gender, smoke, alco, active. Привести матрицу неточностей и сравнить с предыдущими значениями.**

In [5]:
from sklearn.naive_bayes import BernoulliNB

X_binary = df[['gender', 'smoke', 'alco', 'active']]
X_train_b, X_test_b, y_train, y_test = train_test_split(X_binary, y, test_size=0.3, random_state=17)

bnb = BernoulliNB()
y_pred_b = bnb.fit(X_train_b, y_train).predict(X_test_b)

conf_mat_b = confusion_matrix(y_test, y_pred_b)
print(conf_mat_b)

[[8645 1842]
 [8086 2051]]


**Комментарии:** Ваши комментарии здесь.

**4. К этому моменту у вас есть три независимых классификатора: по количественным полям, категориальным и бинарным. Придумать, как их объединить в один единый классификатор, который учитывает все эти поля. Привести матрицу неточностей для него и сравнить с предыдущими значениями. Попытаться объяснить разницу.**

In [8]:
from sklearn.metrics import accuracy_score

# Предыдущее обучение и предсказания для каждого типа данных
# Количественные данные
X_train_q, X_test_q, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)
gnb = GaussianNB()
gnb.fit(X_train_q, y_train)
y_pred_q = gnb.predict(X_test_q)

# Категориальные данные
X_train_c, X_test_c, y_train, y_test = train_test_split(X_cat, y, test_size=0.3, random_state=17)
pred_func = naive_bayes_categorical(X_train_c, y_train)
y_pred_c = X_test_c.apply(pred_func, axis=1)

# Бинарные данные
X_train_b, X_test_b, y_train, y_test = train_test_split(X_binary, y, test_size=0.3, random_state=17)
bnb = BernoulliNB()
bnb.fit(X_train_b, y_train)
y_pred_b = bnb.predict(X_test_b)

# Объединение предсказаний
final_pred = np.round((y_pred_q + y_pred_c + y_pred_b) / 3).astype(int)

# Вычисление матрицы неточностей для финального классификатора
conf_mat_final = confusion_matrix(y_test, final_pred)
accuracy_final = accuracy_score(y_test, final_pred)

print("Матрица неточностей для объединенного классификатора:")
print(conf_mat_final)
print("\nТочность объединенного классификатора:", accuracy_final)

Матрица неточностей для объединенного классификатора:
[[9566  921]
 [7468 2669]]

Точность объединенного классификатора: 0.5932408844065167


**Комментарии:** Ваши комментарии здесь.

**5. (Не обязательно) Теперь мы умеем делать классификацию в рамках наивного предположения об независимости всех признаков. Сейчас же нужно попробовать учесть взаимосвязь между признаками через условные вероятности. Построить классификатор с учетом такой связи. Сравнить результат с ранее полученными значениями.**

In [ ]:
# A lot of code here

**Комментарии:** Ваши комментарии здесь.